In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv

In [2]:
train = pd.read_csv('./train.csv')
keys = ["OutcomeType", "OutcomeSubtype", "AnimalType", "SexuponOutcome", "AgeuponOutcome", "Breed", "Color"]
train

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White


In [3]:
print "OutcomeType"

dict_temp = {}

dict_temp["OutcomeType"] = np.array([val for val in train.OutcomeType.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
dict_temp["OutcomeType"]

OutcomeType


array(['Return_to_owner', 'Euthanasia', 'Adoption', 'Transfer', 'Died'], 
      dtype='|S15')

In [4]:
print "OutcomeSubtype"

dict_temp["OutcomeSubtype"] = np.array([val for val in train.OutcomeSubtype.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])

dict_temp["OutcomeSubtype"] 

OutcomeSubtype


array(['Suffering', 'Foster', 'Partner', 'Offsite', 'SCRP', 'Aggressive',
       'Behavior', 'Rabies Risk', 'Medical', 'In Kennel', 'In Foster',
       'Barn', 'Court/Investigation', 'Enroute', 'At Vet', 'In Surgery'], 
      dtype='|S19')

In [5]:
print "Breed"
dict_temp["Breed"] = np.array([val for val in train.Breed.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
dict_temp["Breed"]

Breed


array(['Shetland Sheepdog Mix', 'Domestic Shorthair Mix', 'Pit Bull Mix',
       ..., 'Vizsla/Boxer', 'German Shepherd/Australian Kelpie',
       'Boxer/German Shepherd'], 
      dtype='|S54')

In [6]:
print "AnimalType"
dict_temp["AnimalType"] = np.array([val for val in train.AnimalType.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
dict_temp["AnimalType"]

AnimalType


array(['Dog', 'Cat'], 
      dtype='|S3')

In [7]:
print "SexuponOutcome"
dict_temp["SexuponOutcome"] = np.array([val for val in train.SexuponOutcome.unique() if str(val) != 'nan' and str(val).lower() != 'unknown'])
dict_temp["SexuponOutcome"]

SexuponOutcome


array(['Neutered Male', 'Spayed Female', 'Intact Male', 'Intact Female'], 
      dtype='|S13')

In [8]:
print "AgeuponOutcome"
dict_temp["AgeuponOutcome"] = train.AgeuponOutcome.unique()
dict_temp["AgeuponOutcome"] 

AgeuponOutcome


array(['1 year', '2 years', '3 weeks', '1 month', '5 months', '4 years',
       '3 months', '2 weeks', '2 months', '10 months', '6 months',
       '5 years', '7 years', '3 years', '4 months', '12 years', '9 years',
       '6 years', '1 weeks', '11 years', '4 weeks', '7 months', '8 years',
       '11 months', '4 days', '9 months', '8 months', '15 years',
       '10 years', '1 week', '0 years', '14 years', '3 days', '6 days',
       '5 days', '5 weeks', '2 days', '16 years', '1 day', '13 years', nan,
       '17 years', '18 years', '19 years', '20 years'], dtype=object)

In [32]:
print "Color"
dict_temp["Color"] = np.array([val for val in train.Color.unique() if str(val) != 'nan' and str(val).lower() != 'unknown'])
dict_temp["Color"]

dict_colors = {}

i = 1

for value in dict_temp["Color"]:
    if value not in dict_colors and '/' not in value:
        dict_colors[value] = i
        i+=1
    elif value not in dict_colors and  '/' in value:
        temp_val = value.split('/')
        temp = '{0}/{1}'.format(temp_val[1], temp_val[0]) 
        if temp not in dict_colors:
            dict_colors[value] = i
            i += 1
            
def getColorCode(colorValue):
    if colorValue in dict_colors:
        return dict_colors[colorValue]
    splits = colorValue.split('/')
    val = '{1}/{0}'.format(splits[0], splits[1])
    return dict_colors[val]

Color


In [10]:
import math

keys

['OutcomeType',
 'OutcomeSubtype',
 'AnimalType',
 'SexuponOutcome',
 'AgeuponOutcome',
 'Breed',
 'Color']

In [11]:
def convertAgeToNum(age):
    if str(age).isdigit():
        return age
    elif str(age) == 'nan':
        return 0
    else:
        age = age.strip()
        age = age.lower()
        splits = age.split()
        if "year" in splits[1]:
            return float(splits[0]) * 365
        elif "month" in splits[1]:
            return float(splits[0]) * 30
        elif "week" in splits[1]:
            return float(splits[0]) * 7
        elif "day" in splits[1]:
            return float(splits[0])

In [34]:
train_transformed = []
for idx, row in train.iterrows():
    lis = []
    for key in keys:
        if key == "AgeuponOutcome":
            age_in_days = convertAgeToNum(row[key])
            lis.append(age_in_days)
        elif key == 'Color':
            if str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
                lis.append(getColorCode(row[key]))
            else:
                lis.append(0)
        elif str(row[key]) != "nan" and str(row[key]).lower() != "unknown":            
            temp = np.where(dict_temp[key] == row[key])[0][0]+1
            lis.append(temp)
        else:
            lis.append(0)
    train_transformed.append(np.array(lis))


In [35]:
train_transformed = np.array(train_transformed)
train_transformed

array([[  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          3.65000000e+02,   1.00000000e+00,   1.00000000e+00],
       [  2.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
          3.65000000e+02,   2.00000000e+00,   2.00000000e+00],
       [  3.00000000e+00,   2.00000000e+00,   1.00000000e+00, ...,
          7.30000000e+02,   3.00000000e+00,   3.00000000e+00],
       ..., 
       [  3.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.46000000e+03,   2.38000000e+02,   2.20000000e+01],
       [  4.00000000e+00,   3.00000000e+00,   2.00000000e+00, ...,
          2.80000000e+01,   2.00000000e+00,   1.10000000e+01],
       [  4.00000000e+00,   5.00000000e+00,   2.00000000e+00, ...,
          3.65000000e+02,   2.00000000e+00,   3.30000000e+01]])